## Import required packages

In [0]:
# Download required packages
!pip -q install gdown missingno torch petastorm wandb pytorch-lightning

%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark import SparkContext, SparkConf
from pyspark.sql.window import Window

from petastorm.spark import SparkDatasetConverter, make_spark_converter
from petastorm.pytorch import DataLoader
spark.conf.set(SparkDatasetConverter.PARENT_CACHE_DIR_URL_CONF, 'file:///dbfs/tmp/petastorm/cache')

import pytorch_lightning as pl
pl.seed_everything(42)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import missingno as msno
import torch
import re
import torch.nn as nn
from typing import *
import datetime
import gdown
import dataclasses
from operator import itemgetter
from functools import partial

import wandb
from pytorch_lightning.loggers import WandbLogger
wandb.login(key='147339090e59f6a02bed0fa3f938a2b1ecbc567c')

import tqdm as tq
def tqdm(*args, **kwargs):
  ''' Small trick to prevent tqdm printing newlines at each step. '''
  return tq.tqdm(*args, **kwargs, leave=True, position=0)

WARNING: You are using pip version 20.2.4; however, version 21.1.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Global seed set to 42
wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
 WARNING: You are using pip version 20.2.4; however, version 21.1.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Global seed set to 42
wandb: Currently logged in as: leonardoemili (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc

## Data aquisition
We retrieve our datasets and download them to a temporary directory in the driver node.

In [0]:
!rm -rf /tmp/data /tmp/__MACOSX
gdown.download('https://drive.google.com/uc?id=1ggmDp-AWFzbQReLG0pLpQE_3fO0C0RnM', '/tmp/data.zip', quiet=False)
!unzip -q /tmp/data.zip -d /tmp/
!rm /tmp/data.zip

Downloading...
From: https://drive.google.com/uc?id=1ggmDp-AWFzbQReLG0pLpQE_3fO0C0RnM
To: /tmp/data.zip
0.00B [00:00, ?B/s]4.72MB [00:00, 18.0MB/s]17.3MB [00:00, 33.2MB/s]25.7MB [00:00, 36.6MB/s]34.1MB [00:00, 38.3MB/s]42.5MB [00:01, 40.9MB/s]58.2MB [00:01, 63.7MB/s]66.6MB [00:01, 52.0MB/s]71.9MB [00:01, 47.7MB/s]

Then we load the datasets to the DBFS.

In [0]:
dbutils.fs.mv("file:/tmp/data", "dbfs:/data", recurse=True)

Out[3]: True

In [0]:
%fs ls /data/

path,name,size
dbfs:/data/.DS_Store,.DS_Store,6148
dbfs:/data/key_stats_yahoo.csv,key_stats_yahoo.csv,2047081
dbfs:/data/prices/,prices/,0


In [0]:
%fs ls /data/

path,name,size
dbfs:/data/.DS_Store,.DS_Store,6148
dbfs:/data/key_stats_yahoo.csv,key_stats_yahoo.csv,2047081
dbfs:/data/prices/,prices/,0


## Dataset loading

In [0]:
key_stats_df = spark.read.load("dbfs:/data/key_stats_yahoo.csv", 
                           format="csv",
                           sep=",",
                           inferSchema="true",
                           header="true"
                          )

# Drop the first ID column
key_stats_df = sc.parallelize(key_stats_df.drop(key_stats_df.columns[0]).head(1005)).toDF()#TODO: remove head(n) (only meant for development)
#key_stats_df = key_stats_df.drop(key_stats_df.columns[0])
key_stats_df.schema['Date'].nullable = False

# Use legacy format to parse dates
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
key_stats_df = key_stats_df.withColumn("Date", F.to_date(key_stats_df["Date"], 'MM/dd/yyyy HH:mm'))

# Cast numerical columns to double
for column in key_stats_df.columns[2:]:
  key_stats_df = key_stats_df.withColumn(column, key_stats_df[column].cast("double"))

# Prices dataframes for every stock #TODO: remove :N (only meant for development)
prices_files = [f.path for f in dbutils.fs.ls('/data/prices/')[:10] if f.path.endswith('.csv')]
#prices_files = [f.path for f in dbutils.fs.ls('/data/prices/') if f.path.endswith('.csv')]
dfs_names = [f.rsplit('/', 1)[1][:-len('.csv')] for f in prices_files]
prices_dfs = []
for f in tqdm(prices_files, desc='Reading stock price data', total=len(prices_files)):
  df = spark.read.load(f,
                       format="csv",
                       sep=",",
                       inferSchema="true",
                       header="true"
                      )
  df = df.withColumn("Date", F.to_date(df["Date"], 'dd-MM-yyyy'))
  df.schema['Date'].nullable = False
  prices_dfs.append(df)

Reading stock price data: 0%| | 0/9 [00:00<?, ?it/s]Reading stock price data: 11%|█ | 1/9 [00:01<00:10, 1.37s/it]Reading stock price data: 22%|██▏ | 2/9 [00:02<00:09, 1.30s/it]Reading stock price data: 33%|███▎ | 3/9 [00:03<00:07, 1.19s/it]Reading stock price data: 44%|████▍ | 4/9 [00:05<00:06, 1.26s/it]Reading stock price data: 56%|█████▌ | 5/9 [00:06<00:04, 1.23s/it]Reading stock price data: 67%|██████▋ | 6/9 [00:07<00:03, 1.18s/it]Reading stock price data: 78%|███████▊ | 7/9 [00:08<00:02, 1.24s/it]Reading stock price data: 89%|████████▉ | 8/9 [00:09<00:01, 1.19s/it]Reading stock price data: 100%|██████████| 9/9 [00:10<00:00, 1.14s/it]Reading stock price data: 100%|██████████| 9/9 [00:10<00:00, 1.20s/it]

## Dataset analysis

In [0]:
print("Prices dataframe format:")
prices_dfs[0].printSchema()

Prices dataframe format:
root
-- Date: date (nullable = true)
-- Low: double (nullable = true)
-- Open: double (nullable = true)
-- Volume: integer (nullable = true)
-- High: double (nullable = true)
-- Close: double (nullable = true)
-- Adjusted Close: double (nullable = true)

In [0]:
print("Key stats dataframe format:")
key_stats_df.printSchema()

Key stats dataframe format:
root
-- Date: date (nullable = true)
-- Ticker: string (nullable = true)
-- Price: double (nullable = true)
-- DE Ratio: double (nullable = true)
-- Trailing P/E: double (nullable = true)
-- Price/Sales: double (nullable = true)
-- Price/Book: double (nullable = true)
-- Profit Margin: double (nullable = true)
-- Operating Margin: double (nullable = true)
-- Return on Assets: double (nullable = true)
-- Return on Equity: double (nullable = true)
-- Revenue Per Share: double (nullable = true)
-- Market Cap: double (nullable = true)
-- Enterprise Value: double (nullable = true)
-- Forward P/E: double (nullable = true)
-- PEG Ratio: double (nullable = true)
-- Enterprise Value/Revenue: double (nullable = true)
-- Enterprise Value/EBITDA: double (nullable = true)
-- Revenue: double (nullable = true)
-- Gross Profit: double (nullable = true)
-- EBITDA: double (nullable = true)
-- Net Income Avl to Common : double (nullable = true)
-- Diluted EPS: double (nullable = true)
-- Earnings Growth: double (nullable = true)
-- Revenue Growth: double (nullable = true)
-- Total Cash: double (nullable = true)
-- Total Cash Per Share: double (nullable = true)
-- Total Debt: double (nullable = true)
-- Current Ratio: double (nullable = true)
-- Book Value Per Share: double (nullable = true)
-- Cash Flow: double (nullable = true)
-- Beta: double (nullable = true)

### Utility functions

In [0]:
# TODO: add remaining utility functions

def prices_df_nan_summary(prices_dfs: List[pyspark.sql.DataFrame], names: List[str]) -> pd.DataFrame:
  ''' Utility function to summarize columns that have missing values. '''
  nan_dfs = []
  for prices_df, name in tqdm(zip(prices_dfs, names), total=len(prices_dfs), desc='Generating prices summary ...'):
    nan_absolute = prices_df.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in prices_df.columns]).first()
    if any(nan_absolute):
      # Simple conversion from Pyspark row -> Python set of values
      values = set(nan_absolute.asDict().values()).difference({0})
      # Either we don't have values for that row, or we have all of them (but Date which is non-nullable)
      # Values contains the no. of NaN values and 0 in correspondance of the Date column
      assert len(values) == 1
      nan_count = values.pop()
      nan_dfs.append((name, round(100*nan_count/prices_df.count(), 3), nan_count))

  return pd.DataFrame(nan_dfs, columns=['Stock name', 'Missing data (%)', 'Count'])

def remove_trailing_nan(df: pyspark.sql.DataFrame, ticker: str, col: str = 'Low') -> pyspark.sql.DataFrame:
  '''
    A trick to detect if the input DataFrame ends with a contiguous collection of NaN rows, returns the dataframe without them.
  '''
  # The total number of rows of the dataframe
  df_length = df.count()
  
  # Sort the input dataframe and add a new column to keep track of the relative position of each row
  df_sorted_id = df.sort('Date').withColumn('id', F.row_number().over(Window.orderBy('Date')))
  
  # Tricky part here: create a new column called 'cumsum' that will store the progressive number of consecutive NaN in our dataset.
  # Let's break it into smaller parts:
  # 1) create an index generator that will partition by 'Low' values [(...,Null...), (...,value1,...), (...,value2...), ... (...)] and within rows order by date
  # Example:
  ## |2019-06-05|null|null|  null|null| null|          null|624|        21|
  ## |2019-06-06|null|null|  null|null| null|          null|625|        22|
  ## |2019-06-07|null|null|  null|null| null|          null|626|        23| <- last column is the cumulative sum (i.e. the number of consecutive NaN)
  ## ...
  ## |2019-05-09|23.2|11.5|   7.3|4.2|  16.2|          29.1|1  |         0|
  ## |2019-05-10|23.2|11.5|   7.3|4.2|  16.2|          29.1|2  |         0|
  # 2) assign to each row a progressive index starting from 1 if it has null in correspondance of Low, zero otherwise
  # 3) store these values into a new column called cumsum (i.e. it behaves like pandas cumsum)
  # 4) at the end, the row whose ID corresponds to the length of the dataframe will contain at column 'cumsum' the no. of trailing NaN values.
  cumsum_df = df_sorted_id.withColumn('cumsum', F.when(F.isnull(df_sorted_id.Low), F.row_number().over(Window.partitionBy('Low').orderBy('Date'))).otherwise(0))

  # Retrieve the "last" row and read the value of cumsum
  end_idx = cumsum_df.where(cumsum_df['id'] == df_length).first().cumsum
  
  # Retain rows whose index is lower len(df) - end_idx + 1 (i.e. cut trailing NaN values)
  return df_sorted_id.where(df_sorted_id['id'] <= df_length-end_idx+1)


def merge_prices_fundamentals(
    prices_dfs: List[pyspark.sql.DataFrame],
    key_stats_df: pyspark.sql.DataFrame,
    dfs_names: List[str],
    drop_cols: List[str] = ['Date', 'Ticker', 'Price']
    ) -> List[pyspark.sql.DataFrame]:
  # Define the target list of dataframes
  prices_dfs_new = []
  for ticker in tqdm(key_stats_df.select('Ticker').distinct().collect(), desc='Merging the datasets ...'):
    ticker = ticker[0]
    # Consider only stocks for which we have fundamental data
    if ticker.upper() not in dfs_names: continue
      
    # The prices dataframe associated to the current ticker
    prices_df_idx = dfs_names.index(ticker.upper())
    prices_df = prices_dfs[prices_df_idx]
    
    # Retrieve financial reports for the current ticker
    ticker_df = key_stats_df.filter(F.col('Ticker') == ticker)
    
    # Perform an inner join between the two dataframes, we first take all reports
    # whose date is at most the prices date (then we will take the latest available)
    joined_df = ticker_df.join(prices_df, ticker_df.Date <= prices_df.Date, how='inner')
    
    # Let's break this into smaller parts:
    # 1) Add a new column for the date latest available report for that stock. To do that
    #    we partition over **prices** Date using the Window object, and the the max over
    #    **ticker** Date.
    # 2) Retain those columns whose financial report date is the same as in ticker_df_max_date
    # 3) Finally drop the **ticker** Date and other unused columns
    joined_df = joined_df.withColumn('ticker_df_max_date', F.max(ticker_df['Date'])\
                      .over(Window.partitionBy(prices_df['Date'])))\
                      .where(ticker_df['Date'] == F.col('ticker_df_max_date'))\
                      .drop('ticker_df_max_date', 'High', 'Low', 'Open', 'Close')\
                      .drop(ticker_df['Date'])\
                      
    # Return the usual list of dataframes
    prices_dfs_new.append(joined_df)
    
  return prices_dfs_new
    
                
def fill_missing_days(
  aggregate_dfs: List[pyspark.sql.DataFrame],
  remove_weekends: bool = True,
  end_date: str = '2014-01-01'
) -> List[pyspark.sql.DataFrame]:
  result_dfs = []
  
  @udf("boolean")
  def is_weekday(date: datetime) -> bool:
      ''' Returns true if the provided date corresponds to a weekday. '''
      return date.weekday() < 5
    
  for df in tqdm(aggregate_dfs, desc='Filling missing days ...'):
    # In this case we apply the following steps in order to apply fill-forward to our dataset:
    # 1) define a Window object the will over the dataset sorted with decreasing dates
    # 2) compute the difference in days between consecutive rows (11/05/2021 - 07/05/2021 ==> 4 days)
    #    and store it in a column named diff
    # 3) compute an increasing index in the column seq that is the days gap we need to fill
    # Example:
    # |Date      | Adjusted close   |diff|seq|new_date  |
    # |2021-04-09|116.80999755859375|   3|  1|2021-04-11|
    # |2021-04-09|116.80999755859375|   3|  2|2021-04-10|
    # |2021-04-09|116.80999755859375|   3|  3|2021-04-09|
    # 4) create a new column new_date as the result of the computation F.col('Date') + F.col('diff') - F.col('seq')
    # 5) only retain rows whose date is before the provided end_date
    #
    # N.B. if we want to bfill the values we should instead consider the dataframes with dates in ascending order,
    # and subtract the value of F.col('diff') instead of summing it. Other parts of the code stays the same.
    df = df\
    .withColumn('diff', F.datediff(F.lag(F.col('Date'),1).over(Window.orderBy(F.desc('Date'))), F.col('Date')))\
    .withColumn('seq', F.explode(F.sequence(F.lit(1), F.col("diff"))))\
    .withColumn('new_date', (F.col('Date') + F.col('diff') - F.col('seq')))\
    .where(F.col('new_date') < pd.Timestamp(end_date))
    
    # If specified, remove weekends that are most likely created by us as synthetic data with the ffill technique
    if remove_weekends:
      df = df.where(is_weekday(F.col('new_date')))
      
    # Drop unecessary columns
    df = df.drop('Date', 'diff', 'seq').withColumnRenamed('new_date', 'Date')
    
    # If the dataframe is actually non-empty, add it to the list of resulting dataframes
    if df.count() > 0:
      result_dfs.append(df)
  return result_dfs

def missing_values_summary(df):
  ''' Returns a utility summary to view missing values in our dataframe. '''
  n = df.count()
  
  def to_percentage(x: pyspark.sql.column.Column, n: int) -> int:
    ''' Utility function to compute the amount of missing values as a percentage of the original dataframe. '''
    return F.round(100 * x / n, 3)
  
  # Aggregate using the count function over null values, and return a view over the obtained (single row) dataframe
  return df.agg(*[to_percentage(F.count(F.when(F.isnull(c), c)), n).alias(c) for c in df.columns]).first()

def scale_features(dfs: List[pyspark.sql.DataFrame]) -> None:
    ''' Scales the numerical features to unit variance. '''
    
    scalable_features = ['DE Ratio', 'Trailing P/E', 'Price/Sales', 'Price/Book',
       'Profit Margin', 'Operating Margin', 'Return on Assets',
       'Return on Equity', 'Revenue Per Share', 'Market Cap',
       'Enterprise Value', 'PEG Ratio', 'Enterprise Value/Revenue',
       'Enterprise Value/EBITDA', 'Revenue', 'Gross Profit', 'EBITDA',
       'Net Income Avl to Common ', 'Diluted EPS', 'Earnings Growth',
       'Revenue Growth', 'Total Cash', 'Total Cash Per Share', 'Total Debt',
       'Current Ratio', 'Book Value Per Share', 'Cash Flow', 'Beta', 'Volume',
       'Adjusted Close', 'SMA', 'RSI']
    

    aggregate_df = dfs[0]
    for df in dfs[1:]:
      aggregate_df = aggregate_df.union(df)
      
    mean = aggregate_df.select(
      [F.mean(F.col(column)) for column in scalable_features]
    ).collect()[0]
    
    std = aggregate_df.select(
      [F.stddev(F.col(column)) for column in scalable_features]
    ).collect()[0]

    
    for i in tqdm(range(len(dfs)), desc='Scaling numerical features ...'):
      # Scaling scalable features
      for j, feature in enumerate(scalable_features):
        dfs[i] = dfs[i].withColumn(feature, (dfs[i][feature]-mean[j])/std[j])
        
def impute_missing_values(
  prices_dfs_new: List[pyspark.sql.DataFrame],
  key_stats_df: pyspark.sql.DataFrame
) -> Tuple[List[pyspark.sql.DataFrame], pyspark.sql.DataFrame]:
  # define the window
  window = Window.orderBy('Date').rowsBetween(Window.unboundedPreceding, 0)

  # Forward filling values 
  # (ref. https://stackoverflow.com/questions/38131982/forward-fill-missing-values-in-spark-python/50422240#50422240)
  for i in range(len(prices_dfs_new)):
    for col_name in prices_dfs_new[i].schema.names:
      col = F.last(prices_dfs_new[i][col_name], ignorenulls=True).over(window)
      prices_dfs_new[i] = prices_dfs_new[i].withColumn(col_name, col)

  # In this case this dataframe contains financial reports that may contain NaN values either because that
  # metric was not available at that time OR because it was monitoring an initial stage of a company growth.
  # What we do is to apply the classic fast-forward, and fill initial missing values with zeroes.
  # Please note: we also discard the 'Forward P/E' column since the imputation here would introduce too much noise.
  key_stats_df_new = key_stats_df.drop('Forward P/E')
  for col_name in key_stats_df_new.schema.names:
      col = F.last(key_stats_df_new[col_name], ignorenulls=True).over(window)
      key_stats_df_new = key_stats_df_new.withColumn(col_name, col)
  key_stats_df_new = key_stats_df_new.fillna(0.)
  
  return prices_dfs_new, key_stats_df_new

def from_dfs(
  dfs: List[pyspark.sql.DataFrame],
  window_size: int = 3,
  target_col: str = 'Adjusted Close',
  val_date: str = '2012-01-01',
  test_date: str = '2013-01-01'
):
  # Split the dataset into train/val/test using these dates as boundaries
  val_begin = pd.Timestamp(val_date)
  test_begin = pd.Timestamp(test_date)
  
  def should_drop(col: pyspark.sql.column, idx: int) -> bool:
    ''' Returns whether the provided column should be dropped. '''
    return (col.endswith(f'_{idx}') or 'Ticker' in col or 'Date' in col or 'id' in col)\
              and col != f'Adjusted Close_{idx}'

  train_split, val_split, test_split = [], [], []
  
  for df in tqdm(dfs, desc='Loading dataset from dfs ...'):
    # Create an ID for each row based on dates
    df = df\
      .sort('Date')\
      .withColumn('id', F.row_number().over(Window.orderBy('Date')))
    
    # Split the stock dataset into train/dev/test splits
    train_df = df.where(F.col('Date') < val_begin)
    val_df = df.where(F.col('Date').between(val_begin, test_begin-pd.Timedelta(days=1)))
    test_df = df.where(F.col('Date') >= test_begin)
    
    # Add a unique identifier for each column (e.g. Date -> Date_0, Date_1, etc ...)
    train_df = prepend_columns_index(train_df)
    val_df = prepend_columns_index(val_df)
    test_df = prepend_columns_index(test_df)
    
    # Convert each split from nxm -> nxm*(w+1), where w is the size of the window
    train_df = create_window_indexes(train_df, window_size)
    val_df = create_window_indexes(val_df, window_size)
    test_df = create_window_indexes(test_df, window_size)
    
    # Collect the processed datasets
    train_split.append(train_df)
    val_split.append(val_df)
    test_split.append(test_df)
  
  # Initialize the final datasets using the obtained df schema
  train_ds = spark.createDataFrame([], train_split[0].schema)
  val_ds = spark.createDataFrame([], val_split[0].schema)
  test_ds = spark.createDataFrame([], test_split[0].schema)
  
  # Collect all the dataframes into a single df for each data split
  for df in train_split: train_ds = train_ds.union(df)
  for df in val_split: val_ds = val_ds.union(df)
  for df in test_split: test_ds = test_ds.union(df)
  
  # Drop unused columns for the target variable and rename the target column as 'y'
  drop_cols = [col for col in test_ds.columns if should_drop(col, window_size)]
  train_ds = train_ds.drop(*drop_cols).withColumnRenamed(f'Adjusted Close_{window_size}', 'y')
  val_ds = val_ds.drop(*drop_cols).withColumnRenamed(f'Adjusted Close_{window_size}', 'y')
  test_ds = test_ds.drop(*drop_cols).withColumnRenamed(f'Adjusted Close_{window_size}', 'y')
  
  # Rename columns to make them compatible with Apache Parquet format
  target_cols = list(map(lambda x: '_'.join(re.split(' |/', x)), train_ds.columns))
  for c, target in zip(train_ds.columns, target_cols): train_ds = train_ds.withColumnRenamed(c, target)
  for c, target in zip(val_ds.columns, target_cols): val_ds = val_ds.withColumnRenamed(c, target)
  for c, target in zip(test_ds.columns, target_cols): test_ds = test_ds.withColumnRenamed(c, target)
    
  # Compute the set of unique features in each split
  features = list(set(c[:-2] for c in train_ds.columns if len(c) >= 2))

  # Convert each split to have, at each timestep (column), the collection of features at that timestep
  for timestep in range(window_size):
    # Consider all the features in this timestep
    current_cols = list(map(lambda c: f'{c}_{timestep}', features))
    # Convert (price_0, sma_0, market_cap_0) -> 0: (price|sma|market_cap) 
    train_ds = train_ds.withColumn(f'x_{timestep}', F.array(current_cols)).drop(*current_cols)
    val_ds = val_ds.withColumn(f'x_{timestep}', F.array(current_cols)).drop(*current_cols)
    test_ds = test_ds.withColumn(f'x_{timestep}', F.array(current_cols)).drop(*current_cols)
  
  return train_ds, val_ds, test_ds, features

def prepend_columns_index(
  df: pyspark.sql.DataFrame,
  idx: int = 0
) -> pyspark.sql.DataFrame:
  ''' Utility function that returns a new dataframe where columns are preceeded by [idx]. '''
  for col in df.schema.names:
    # If this is not the first time this function is applied, simply edit the previous identifier
    new_col = f'{col}_{idx}' if idx == 0 else f'{col[:-2]}_{idx}'
    df = df.withColumnRenamed(col, new_col)
  return df

def create_window_indexes(df: pyspark.sql.DataFrame, window_size: int) -> pyspark.sql.DataFrame:
  ''' Apply self join multiple times to the input dataframe, return a new one of size (df.count(), len(df.columns)*window_size) '''
  joined_df = df
  for win_idx in range(1, window_size+1):
    joined_df = joined_df.join(
      prepend_columns_index(df, win_idx),
      F.col('id_0') == F.col(f'id_{win_idx}') - win_idx
    )
  return joined_df

def collate_fn(
  time_steps: List[str],
  batch: List[Dict[str, np.array]]
) -> Tuple[torch.Tensor, torch.Tensor]:
  ''' Returns (x, y) pairs where x is a vector of shape: (window_size, feature_size). '''
  X = np.asarray([np.stack(itemgetter(*time_steps)(d)) for d in batch])
  Y = np.asarray([d['y'] for d in batch])
  return torch.from_numpy(X).double(), torch.from_numpy(Y).double()

def data_loader_fn(window_size: int, **kwargs) -> DataLoader:
  ''' Override default behaviour and return a petastorm.Dataset with custom collate function. '''
  time_steps = [f'x_{time_step}' for time_step in range(window_size)]
  return DataLoader(**kwargs, collate_fn=partial(collate_fn, time_steps))

def r2_score(y_hat: np.ndarray, y: np.ndarray) -> float:
    ''' Computes the R Squared coefficient between y_hat and y. '''
    rss = torch.sum((y - y_hat) ** 2)
    tss = torch.sum((y - y.mean()) ** 2)
    return 1 - rss / tss

def adjusted_r2_score(y_hat: np.ndarray, y: np.ndarray, p: int) -> float:
    ''' Computes the Adjusted R Squared coefficient between y_hat and y. '''
    rss = torch.sum((y - y_hat) ** 2)
    tss = torch.sum((y - y.mean()) ** 2)
    df_e = y.shape[0] - p - 1
    df_t = y.shape[0] - 1
    return 1 - (rss / tss) * (df_t / df_e)

#### Technical indicators

In [0]:
def add_sma(dfs: List[pyspark.sql.DataFrame], period: int = 10) -> None:
    ''' Computes the Simple Moving Average from a given dataframe. '''
    w = (Window().partitionBy("Adjusted Close").orderBy(F.col("Date").cast('long')).rangeBetween(-period*86400, 0))
    #w = Window().partitionBy("Adjusted Close").orderBy(F.col("Date"))
    for i in tqdm(range(len(dfs)), desc='Adding SMA ...'):
        dfs[i] = dfs[i].withColumn(
          "SMA",
          F.avg("Adjusted Close").over(w)
        )

        
def add_rsi(dfs: List[pyspark.sql.DataFrame], period: int = 14) -> None:
    ''' Computes the Relative Strength Index from a given dataframe. 
        Formula available at https://en.wikipedia.org/wiki/Relative_strength_index.
        Also adds overbought and oversold when the RSI index hits 70 or 30.'''
    for j in tqdm(range(len(dfs)), desc='Adding RSI ...'):
        
        dateCol = dfs[j]["Date"]
      
        w = Window.partitionBy().orderBy("Date")

        dfs[j] = dfs[j].withColumn("prev_adj_close", F.lag(dfs[j]["Adjusted Close"]).over(w))
        dfs[j] = dfs[j].withColumn("current_gain", F.when(dfs[j]["Adjusted Close"] >= dfs[j]["prev_adj_close"],
                                                          dfs[j]["Adjusted Close"] - dfs[j]["prev_adj_close"])
                                                          .otherwise(0.0))
        dfs[j] = dfs[j].withColumn("current_loss", F.when(dfs[j]["prev_adj_close"] >= dfs[j]["Adjusted Close"],
                                                         dfs[j]["prev_adj_close"] - dfs[j]["Adjusted Close"])
                                                         .otherwise(0.0))
          
        w = (Window().partitionBy("current_gain").orderBy(F.col("Date").cast('long')).rangeBetween(-period*86400, 0))
        dfs[j] = dfs[j].withColumn(
          "smmau",
          F.avg("current_gain").over(w)
        )
        
        w = (Window().partitionBy("current_loss").orderBy(F.col("Date").cast('long')).rangeBetween(-period*86400, 0))
        dfs[j] = dfs[j].withColumn(
          "smmad",
          F.avg("current_loss").over(w)
        )
        
        dfs[j] = dfs[j].withColumn("RSI", 100 - (100/(1+dfs[j]["smmau"]/dfs[j]["smmad"])))                        
        dfs[j] = dfs[j].withColumn("Overbought", F.when(dfs[j]["RSI"] >= 70, 1.0).otherwise(0.0))
        dfs[j] = dfs[j].withColumn("Oversold", F.when(dfs[j]["RSI"] <= 30, 1.0).otherwise(0.0))
        dfs[j] = dfs[j].withColumn("Date", dateCol)
        
        dfs[j] = dfs[j].drop("prev_adj_close", "current_gain", "current_loss", "smmau", "smmad")

In [0]:
print("Overview of the missing values in the key_stats dataframe\n")
key_stats_summary = missing_values_summary(key_stats_df)
key_stats_summary

Overview of the missing values in the key_stats dataframe

Out[13]: Row(Date=0.0, Ticker=0.0, Price=1.99, DE Ratio=18.607, Trailing P/E=9.652, Price/Sales=0.498, Price/Book=2.189, Profit Margin=1.194, Operating Margin=0.498, Return on Assets=1.99, Return on Equity=2.786, Revenue Per Share=0.398, Market Cap=0.199, Enterprise Value=1.095, Forward P/E=46.269, PEG Ratio=4.776, Enterprise Value/Revenue=1.294, Enterprise Value/EBITDA=7.96, Revenue=1.294, Gross Profit=2.886, EBITDA=7.662, Net Income Avl to Common =0.1, Diluted EPS=0.995, Earnings Growth=17.015, Revenue Growth=1.194, Total Cash=2.189, Total Cash Per Share=1.095, Total Debt=8.159, Current Ratio=6.567, Book Value Per Share=0.199, Cash Flow=11.542, Beta=2.886)

### Missing values imputation

In [0]:
summary = prices_df_nan_summary(prices_dfs, dfs_names)
px.bar(summary, x='Stock name', y='Missing data (%)', hover_data=['Count'], title="Stock price dataset before preprocessing (only columns with missing values are displayed)")

Generating prices summary ...: 0%| | 0/9 [00:00<?, ?it/s]Generating prices summary ...: 11%|█ | 1/9 [00:01<00:10, 1.34s/it]Generating prices summary ...: 22%|██▏ | 2/9 [00:02<00:07, 1.09s/it]Generating prices summary ...: 33%|███▎ | 3/9 [00:03<00:05, 1.01it/s]Generating prices summary ...: 44%|████▍ | 4/9 [00:04<00:06, 1.31s/it]Generating prices summary ...: 56%|█████▌ | 5/9 [00:05<00:04, 1.13s/it]Generating prices summary ...: 67%|██████▋ | 6/9 [00:06<00:03, 1.06s/it]Generating prices summary ...: 78%|███████▊ | 7/9 [00:07<00:01, 1.00it/s]Generating prices summary ...: 89%|████████▉ | 8/9 [00:08<00:01, 1.08s/it]Generating prices summary ...: 100%|██████████| 9/9 [00:09<00:00, 1.05s/it]Generating prices summary ...: 100%|██████████| 9/9 [00:09<00:00, 1.09s/it]
Out[14]:

For most of the above stocks with missing values, we noticed that they indeed exist up to a given time and after that no more data is available. It may due to a business failure, hence no more stocks will be exchanged from that moment on.

In [0]:
# Clear our input data from training NaN values
prices_dfs_new = [remove_trailing_nan(df,name) for df,name in tqdm(zip(prices_dfs, dfs_names), total=len(prices_dfs), desc='Removing trailing NaN values ...')]

# Remove INTH stock from our dataset since it contains many inactivity periods
#inth_idx = dfs_names.index('INTH') #TODO: uncomment
#del dfs_names[inth_idx] #TODO: uncomment
#del prices_dfs_new[inth_idx] #TODO: uncomment

summary = prices_df_nan_summary(prices_dfs_new, dfs_names)
px.bar(summary, x='Stock name', y='Missing data (%)', hover_data=['Count'], title="Stock price dataset after preprocessing (only columns with missing values are displayed)")

Removing trailing NaN values ...: 0%| | 0/9 [00:00<?, ?it/s]Removing trailing NaN values ...: 11%|█ | 1/9 [00:03<00:28, 3.62s/it]Removing trailing NaN values ...: 22%|██▏ | 2/9 [00:05<00:19, 2.75s/it]Removing trailing NaN values ...: 33%|███▎ | 3/9 [00:07<00:14, 2.37s/it]Removing trailing NaN values ...: 44%|████▍ | 4/9 [00:10<00:12, 2.60s/it]Removing trailing NaN values ...: 56%|█████▌ | 5/9 [00:12<00:09, 2.49s/it]Removing trailing NaN values ...: 67%|██████▋ | 6/9 [00:15<00:07, 2.44s/it]Removing trailing NaN values ...: 78%|███████▊ | 7/9 [00:17<00:04, 2.29s/it]Removing trailing NaN values ...: 89%|████████▉ | 8/9 [00:19<00:02, 2.18s/it]Removing trailing NaN values ...: 100%|██████████| 9/9 [00:20<00:00, 2.02s/it]Removing trailing NaN values ...: 100%|██████████| 9/9 [00:20<00:00, 2.32s/it]
Generating prices summary ...: 0%| | 0/9 [00:00<?, ?it/s]Generating prices summary ...: 11%|█ | 1/9 [00:01<00:12, 1.59s/it]Generating prices summary ...: 22%|██▏ | 2/9 [00:02<00:10, 1.48s/it]Generating prices summary ...: 33%|███▎ | 3/9 [00:04<00:09, 1.52s/it]Generating prices summary ...: 44%|████▍ | 4/9 [00:08<00:11, 2.34s/it]Generating prices summary ...: 56%|█████▌ | 5/9 [00:09<00:08, 2.11s/it]Generating prices summary ...: 67%|██████▋ | 6/9 [00:11<00:05, 1.92s/it]Generating prices summary ...: 78%|███████▊ | 7/9 [00:12<00:03, 1.71s/it]Generating prices summary ...: 89%|████████▉ | 8/9 [00:14<00:01, 1.64s/it]Generating prices summary ...: 100%|██████████| 9/9 [00:15<00:00, 1.49s/it]Generating prices summary ...: 100%|██████████| 9/9 [00:15<00:00, 1.70s/it]
Out[15]:

At this point we use the fast forward imputation technique to fill-in missing values. Please note that in this case missing values are mostly due to holidays or periods when stocks are not exchanged.

## Hyperparameters

In [0]:
@dataclasses.dataclass
class HParams:
  input_dim: int = -1  # Denoted by data transformations
  window_size: int = 5
  batch_size: int = 1024
  features: int = -1  # Denoted by data transformations
  hidden_dim: int = 128
  num_layers: int = 1
  dropout: int = 0.5
  lr: int = 0.1
    
hparams = HParams()

## Building our new dataset

In [0]:
# Impute missing values in the prices dataset (i.e. fast-forward last valid values)
prices_dfs_new, key_stats_df_new = impute_missing_values(prices_dfs_new, key_stats_df)

# Merge the stock price dataset with fundamental data of the relative company
aggregate_dfs = merge_prices_fundamentals(prices_dfs_new, key_stats_df_new, dfs_names)

# Fill gaps from the original dataset
dfs = fill_missing_days(aggregate_dfs)
                  
# Add SMA indicator to each dataframe
add_sma(dfs)

# Add RSI indicator to each dataframe
add_rsi(dfs)

# Scale numerical features
scale_features(dfs)

# Create train/dev/test splits
train_ds, val_ds, test_ds, features = from_dfs(dfs, window_size=hparams.window_size)
hparams.input_dim = len(features)
hparams.features = features

Merging the datasets ...: 0%| | 0/56 [00:00<?, ?it/s]Merging the datasets ...: 18%|█▊ | 10/56 [00:00<00:01, 31.18it/s]Merging the datasets ...: 32%|███▏ | 18/56 [00:00<00:01, 36.88it/s]Merging the datasets ...: 39%|███▉ | 22/56 [00:00<00:01, 32.05it/s]Merging the datasets ...: 54%|█████▎ | 30/56 [00:00<00:00, 38.72it/s]Merging the datasets ...: 79%|███████▊ | 44/56 [00:00<00:00, 57.03it/s]Merging the datasets ...: 100%|██████████| 56/56 [00:01<00:00, 62.58it/s]Merging the datasets ...: 100%|██████████| 56/56 [00:01<00:00, 49.74it/s]
Filling missing days ...: 0%| | 0/6 [00:00<?, ?it/s]Filling missing days ...: 17%|█▋ | 1/6 [00:05<00:27, 5.57s/it]Filling missing days ...: 33%|███▎ | 2/6 [00:08<00:17, 4.31s/it]Filling missing days ...: 50%|█████ | 3/6 [00:13<00:12, 4.22s/it]Filling missing days ...: 67%|██████▋ | 4/6 [00:16<00:07, 3.96s/it]Filling missing days ...: 83%|████████▎ | 5/6 [00:19<00:03, 3.56s/it]Filling missing days ...: 100%|██████████| 6/6 [00:21<00:00, 3.11s/it]Filling missing days ...: 100%|██████████| 6/6 [00:21<00:00, 3.63s/it]
Adding SMA ...: 0%| | 0/6 [00:00<?, ?it/s]Adding SMA ...: 33%|███▎ | 2/6 [00:00<00:00, 12.01it/s]Adding SMA ...: 83%|████████▎ | 5/6 [00:00<00:00, 17.66it/s]Adding SMA ...: 100%|██████████| 6/6 [00:00<00:00, 17.92it/s]
Loading dataset from dfs ...: 0%| | 0/6 [00:00<?, ?it/s]Loading dataset from dfs ...: 17%|█▋ | 1/6 [00:08<00:41, 8.38s/it]Loading dataset from dfs ...: 33%|███▎ | 2/6 [00:14<00:29, 7.32s/it]Loading dataset from dfs ...: 50%|█████ | 3/6 [00:21<00:21, 7.03s/it]Loading dataset from dfs ...: 67%|██████▋ | 4/6 [00:28<00:13, 6.84s/it]Loading dataset from dfs ...: 83%|████████▎ | 5/6 [00:35<00:06, 6.84s/it]Loading dataset from dfs ...: 100%|██████████| 6/6 [00:41<00:00, 6.64s/it]Loading dataset from dfs ...: 100%|██████████| 6/6 [00:41<00:00, 6.88s/it]

##Models definition

In [0]:
class SimpleLSTM(nn.Module):
  def __init__(self, hparams):
      super().__init__()
      self.hparams = hparams
      self.lstm = nn.LSTM(hparams.input_dim, hparams.hidden_dim, num_layers=hparams.num_layers, batch_first=True)
      self.fc1 = nn.Linear(hparams.hidden_dim, hparams.hidden_dim//2)
      self.fc2 = nn.Linear(hparams.hidden_dim//2, 1)
      self.dropout = nn.Dropout(hparams.dropout)

  def forward(self, x: np.ndarray):
      x, (h,c) = self.lstm(x)
      h = self.dropout(h)
      h = h.view(self.hparams.num_layers, -1, self.hparams.hidden_dim)[-1]
      x = torch.relu(self.fc1(h))
      x = self.fc2(x)
      return x.squeeze()

class StockPriceRegressor(pl.LightningModule):
  def __init__(self, hparams: HParams):
      super().__init__()
      self.save_hyperparameters(dataclasses.asdict(hparams))
      self.loss_fn = nn.MSELoss()
      self.model = SimpleLSTM(self.hparams)

  def training_step(self, batch: Tuple[np.ndarray, np.ndarray], batch_idx: int):
      x, y = batch
      y_hat = self.model(x)
      loss = self.loss_fn(y_hat, y)
      return loss

  @torch.no_grad()
  def evaluation(self, batch: Tuple[np.ndarray, np.ndarray]) -> float:
      x, y = batch
      y_hat = self.model(x)
      loss = self.loss_fn(y_hat, y)
      r2 = r2_score(y_hat, y)
      # p := no. of explanatory variables in the model (i.e window_size * feature_size)
      r2_adj = adjusted_r2_score(y_hat, y, x.shape[1] * x.shape[2])
      return loss, r2, r2_adj

  def validation_step(self, batch: Tuple[np.ndarray, np.ndarray], batch_idx: int) -> None:
      loss, r2, r2_adj = self.evaluation(batch)
      metrics = {'val_loss': loss, 'val_r2': r2, 'val_r2_adj': r2_adj}
      self.log_dict(metrics)

  def test_step(self, batch: Tuple[np.ndarray, np.ndarray], batch_idx: int) -> None:
      loss, r2, r2_adj = self.evaluation(batch)
      metrics = {'test_loss': loss, 'test_r2': r2, 'val_r2_adj': r2_adj}
      self.log_dict(metrics)

  def configure_optimizers(self):
      return torch.optim.SGD(self.parameters(), lr=self.hparams.lr)

In [0]:
# Define the number of epochs to run the model on
hparams.epochs = 5

# Configure the current run
run = wandb.init(reinit=True, project='distributed_spf')
model = StockPriceRegressor(hparams).double()
trainer = pl.Trainer(
  logger = WandbLogger(),
  #accelerator='ddp',
  log_every_n_steps = 1,
  max_epochs = hparams.epochs,
  deterministic = True
)

# Materialize spark dataframes to DBFS in parqet format
converter_train = make_spark_converter(train_ds)
converter_valid = make_spark_converter(val_ds)

# Fit the model on training data, and evaluate it on validation split
with converter_train.make_torch_dataloader(
  batch_size=hparams.batch_size, data_loader_fn=partial(data_loader_fn, hparams.window_size), num_epochs=hparams.epochs
) as train_dataloader, converter_valid.make_torch_dataloader(
  batch_size=hparams.batch_size, data_loader_fn=partial(data_loader_fn, hparams.window_size), num_epochs=hparams.epochs
) as valid_dataloader:
  
  trainer.fit(model, train_dataloader, valid_dataloader)

# Log the experiments
run.finish()

<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
The median size 689922 B (< 50 MB) of the parquet files is too small. Total size: 3280858 B. Increase the median file size by calling df.repartition(n) or df.coalesce(n), which might help improve the performance. Parquet files: file:/dbfs/tmp/petastorm/cache/20210514200755-appid-local-1621021446187-88f09c03-c27e-4c96-87f7-3efb965363aa/part-00000-tid-7215225489191546835-6d23c333-065b-4796-ab55-3ab9ec160abc-426-1-c000.parquet, ...
The median size 70347 B (< 50 MB) of the parquet files is too small. Total size: 347810 B. Increase the median file size by calling df.repartition(n) or df.coalesce(n), which might help improve the performance. Parquet files: file:/dbfs/tmp/petastorm/cache/20210514205237-appid-local-1621021446187-0803381f-d2d4-4efe-9a34-e5ddb0f5d9bd/part-00000-tid-6687446883553154058-6878712f-694b-4b7a-b4e6-1a10ed274d3f-940-1-c000.parquet, ...

 Name | Type | Params
---------------------------------------
0 | loss_fn | MSELoss | 0 
1 | model | SimpleLSTM | 91.3 K
---------------------------------------
91.3 K Trainable params
0 Non-trainable params
91.3 K Total params
0.365 Total estimated model params size (MB)
Validation sanity check: 0it [00:00, ?it/s]Validation sanity check: 0%| | 0/2 [00:00<?, ?it/s]Validation sanity check: 50%|█████ | 1/2 [00:00<00:00, 4.76it/s]Validation sanity check: 100%|██████████| 2/2 [00:00<00:00, 4.66it/s] Global seed set to 42
Training: 0it [00:00, ?it/s]Training: 0it [00:00, ?it/s]Epoch 0: : 0it [00:00, ?it/s]Epoch 0: : 1it [00:00, 2.70it/s]Epoch 0: : 1it [00:00, 2.61it/s, loss=242, v_num=jvem]Epoch 0: : 2it [00:00, 2.84it/s, loss=242, v_num=jvem]Epoch 0: : 2it [00:00, 2.81it/s, loss=161, v_num=jvem]Epoch 0: : 3it [00:00, 3.01it/s, loss=161, v_num=jvem]Epoch 0: : 3it [00:01, 2.99it/s, loss=4.98e+03, v_num=jvem]Epoch 0: : 4it [00:01, 3.20it/s, loss=4.98e+03, v_num=jvem]Epoch 0: : 4it [00:01, 3.07it/s, loss=4.07e+03, v_num=jvem]Epoch 0: : 5it [00:01, 3.15it/s, loss=4.07e+03, v_num=jvem]Epoch 0: : 5it [00:01, 3.14it/s, loss=1.05e+07, v_num=jvem]Epoch 0: : 6it [00:01, 3.26it/s, loss=1.05e+07, v_num=jvem]Epoch 0: : 6it [00:01, 3.25it/s, loss=7.23e+17, v_num=jvem]Epoch 0: : 7it [00:02, 3.34it/s, loss=7.23e+17, v_num=jvem]Epoch 0: : 7it [00:02, 3.33it/s, loss=3.76e+33, v_num=jvem]Epoch 0: : 8it [00:02, 3.31it/s, loss=3.76e+33, v_num=jvem]Epoch 0: : 8it [00:02, 3.30it/s, loss=inf, v_num=jvem] Epoch 0: : 9it [00:02, 3.34it/s, loss=inf, v_num=jvem]Epoch 0: : 9it [00:02, 3.33it/s, loss=inf, v_num=jvem]Epoch 0: : 10it [00:02, 3.34it/s, loss=inf, v_num=jvem]Epoch 0: : 10it [00:03, 3.33it/s, loss=inf, v_num=jvem]Epoch 0: : 11it [00:03, 3.37it/s, loss=inf, v_num=jvem]Epoch 0: : 11it [00:03, 3.36it/s, loss=nan, v_num=jvem]Epoch 0: : 12it [00:03, 3.36it/s, loss=nan, v_num=jvem]Epoch 0: : 12it [00:03, 3.35it/s, loss=nan, v_num=jvem]Epoch 0: : 13it [00:03, 3.38it/s, loss=nan, v_num=jvem]Epoch 0: : 13it [00:03, 3.38it/s, loss=nan, v_num=jvem]Epoch 0: : 14it [00:04, 3.36it/s, loss=nan, v_num=jvem]Epoch 0: : 14it [00:04, 3.36it/s, loss=nan, v_num=jvem]Epoch 0: : 15it [00:04, 3.39it/s, loss=nan, v_num=jvem]Epoch 0: : 15it [00:04, 3.38it/s, loss=nan, v_num=jvem]Epoch 0: : 16it [00:04, 3.37it/s, loss=nan, v_num=jvem]Epoch 0: : 16it [00:04, 3.37it/s, loss=nan, v_num=jvem]Epoch 0: : 17it [00:05, 3.39it/s, loss=nan, v_num=jvem]Epoch 0: : 17it [00:05, 3.39it/s, loss=nan, v_num=jvem]Epoch 0: : 18it [00:05, 3.38it/s, loss=nan, v_num=jvem]Epoch 0: : 18it [00:05, 3.37it/s, loss=nan, v_num=jvem]Epoch 0: : 19it [00:05, 3.36it/s, loss=nan, v_num=jvem]Epoch 0: : 19it [00:05, 3.35it/s, loss=nan, v_num=jvem]Epoch 0: : 20it [00:05,

--------------------------------------------------------------------------- 
 NotImplementedError Traceback (most recent call last)
 <command-3754394103994545> in <module> 
 20 ) as valid_dataloader:
 21 
 ---> 22 trainer . fit ( model , train_dataloader , valid_dataloader ) 
 23 
 24 run . finish ( ) 

 /databricks/python/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py in fit (self, model, train_dataloader, val_dataloaders, datamodule) 
 456 )
 457 
 --> 458 self . _run ( model ) 
 459 
 460 assert self . state . stopped

 /databricks/python/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py in _run (self, model) 
 754 
 755 # dispatch `start_training` or `start_evaluating` or `start_predicting` 
 --> 756 self . dispatch ( ) 
 757 
 758 # plugin will finalized fitting (e.g. ddp_spawn will load trained model) 

 /databricks/python/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py in dispatch (self) 
 795 self . accelerator . start_predicting ( self ) 
 796 else : 
 --> 797 self . accelerator . start_training ( self ) 
 798 
 799 def run_stage ( self ) : 

 /databricks/python/lib/python3.8/site-packages/pytorch_lightning/accelerators/accelerator.py in start_training (self, trainer) 
 94 
 95 def start_training ( self , trainer : 'pl.Trainer' ) -> None : 
 ---> 96 self . training_type_plugin . start_training ( trainer ) 
 97 
 98 def start_evaluating ( self , trainer : 'pl.Trainer' ) -> None : 

 /databricks/python/lib/python3.8/site-packages/pytorch_lightning/plugins/training_type/training_type_plugin.py in start_training (self, trainer) 
 142 def start_training ( self , trainer : 'pl.Trainer' ) -> None : 
 143 # double dispatch to initiate the training loop 
 --> 144 self . _results = trainer . run_stage ( ) 
 145 
 146 def start_evaluating ( self , trainer : 'pl.Trainer' ) -> None : 

 /databricks/python/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py in run_stage (self) 
 805 if self . predicting : 
 806 return self . run_predict ( ) 
 --> 807 return self . run_train ( ) 
 808 
 809 def _pre_training_routine ( self ) : 

 /databricks/python/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py in run_train (self) 
 867 with self . profiler . profile ( "run_training_epoch" ) : 
 868 # run train epoch 
 --> 869 self . train_loop . run_training_epoch ( ) 
 870 
 871 if self . max_steps and self . max_steps <= self . global_step : 

 /databricks/python/lib/python3.8/site-packages/pytorch_lightning/trainer/training_loop.py in run_training_epoch (self) 
 574 if should_check_val : 
 575 self . trainer . validating = True 
 --> 576 self . trainer . run_evaluation ( on_epoch = True ) 
 577 self . trainer . training = True 
 578 

 /databricks/python/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py in run_evaluation (self, on_epoch) 
 947 dl_max_batches = self . evaluation_loop . max_batches [ dataloader_idx ] 
 948 
 --> 949 for batch_idx , batch in enumerate ( dataloader ) : 
 950 if batch is None : 
 951 continue 

 /databricks/python/lib/python3.8/site-packages/petastorm/pytorch.py in __iter__ (self) 
 114 raise RuntimeError ( _PARALLEL_ITER_ERROR ) 
 115 if self . _in_iter is not None : 
 --> 116 self . reader . reset ( ) 
 117 logger . warning ( 'Start a new pass of Petastorm DataLoader, reset underlying Petastorm reader to position 0.' ) 
 118 self . _in_iter = True 

 /databricks/python/lib/python3.8/site-packages/petastorm/reader.py in reset (self) 
 492 # drop these in-flight samples? Or just ignore it? What would happen if we have two concurrent ventilators 
 493 # that are emitting load requests at the same time? 
 --> 494 raise NotImplementedError('Currently do not support resetting a reader while in the middle of iteration. '
 495 'You can call reset only after all samples were consumed.')
 496 self . last_row_consumed = False 

 NotImplementedError : Currently do not support resetting a reader while in the middle of iteration. You 